# Terminator efficiency

Use Ju et al data to test how term efficiency depends on the promoter strength

* Input: SeqEnd wig files and Excel file with terminator coords and props
* Output: dataset of terminators with termination efficiencies, read-through rates; correlation graphs etc.

# Prepare


## Install modules

In [ ]:
!pip install openpyxl --upgrade --pre
!pip3 install gffpandas
!python -m pip install -U mpltern
!pip install pingouin

## Import modules

In [ ]:
import os
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import copy
from sys import path as syspath
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import pathlib
from random import randint, seed
import gzip
import gffpandas.gffpandas as gffpd
import mpltern
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor

## Select dataset for the analysis

In [ ]:
experiment_set = 'new_primary_transcipts'

term_mechanism = 'Rho'#'intrinsic'#

if term_mechanism == 'Rho':
    term_mech_prefix = 'Rho'
elif term_mechanism == 'intrinsic':
    term_mech_prefix = 'IT'
    
is_bidir = False # Are we looking at bidirectional only (True) or unidirctional promoters only (False)
if is_bidir:
    bidir_prefix = 'BI'
else:
    bidir_prefix = 'UNI'

## Setup paths

In [ ]:
# Working dir
cwd = pathlib.Path(os.getcwd())
print(cwd)

# Input folder
input_folderpath = cwd.parents[2] / 'input/data/2023-04_18_from_Ju_et_al/GSE117737_RAW'
print(input_folderpath)
assert input_folderpath.is_dir()

# Input terminators excel file
input_xls_terms_filename = input_folderpath.parents[0] / '41564_2019_500_MOESM4_ESM.xlsx'
assert input_xls_terms_filename.is_file()

# Input BIDIRECTIONAL terminators excel file
input_xls_bidir_terms_filename = input_folderpath.parents[0] / '41564_2019_500_MOESM6_ESM.xlsx'
assert input_xls_bidir_terms_filename.is_file()

# Output files
plots_out_folder = cwd.parents[2] / 'output/plots'
final_excel_table = str(cwd.parents[2]) + '/output/output_data/'+bidir_prefix+'_'+term_mechanism+'_final_df.xlsx'
print(final_excel_table)
Ju_full_excel_table = cwd.parents[2] / 'output/output_data/Ju_full_data.xlsx'
all_Ju_terms_fasta_file_path = cwd.parents[2] / 'in_progress/processed_data/all_Ju_terms.fasta'

# Experiments
    
if experiment_set == 'new_primary_transcipts':
    processed_data_folderpath = cwd.parents[2] / 'in_progress/processed_data/2023-12-01_Primary_transcripts'
    phases = ['log', 'stat']
    strands = ['plus', 'minus']
    log_reps = ['rep_1','rep_2','rep_3','rep_4','rep_5']
    stat_reps = ['rep_1','rep_2','rep_3']

    sign_strand = {}
    sign_strand['plus'] = '+'
    sign_strand['minus'] = '-'

    # Input SeqEnd wig files
    input_filepaths = {}

    ## Log and stationary phases
    input_filepaths['log'] = {}
    input_filepaths['stat'] = {}

    ## Plus & minus strands
    input_filepaths['log']['plus'] = {}
    input_filepaths['log']['minus'] = {}

    input_filepaths['stat']['plus'] = {}
    input_filepaths['stat']['minus'] = {}

    ## Replicates

    ### Log phase
    #### Plus strand
    input_filepaths['log']['plus']['rep_1'] = processed_data_folderpath / 'Log_phase/Log_rep1_plus.wig.gz'
    assert input_filepaths['log']['plus']['rep_1'].is_file()

    input_filepaths['log']['plus']['rep_2'] = processed_data_folderpath / 'Log_phase/Log_rep2_plus.wig.gz'
    assert input_filepaths['log']['plus']['rep_2'].is_file()
    
    input_filepaths['log']['plus']['rep_3'] = processed_data_folderpath / 'Log_phase/Log_rep3_plus.wig.gz'
    assert input_filepaths['log']['plus']['rep_3'].is_file()
    
    input_filepaths['log']['plus']['rep_4'] = processed_data_folderpath / 'Log_phase/Log_rep4_plus.wig.gz'
    assert input_filepaths['log']['plus']['rep_4'].is_file()
    
    input_filepaths['log']['plus']['rep_5'] = processed_data_folderpath / 'Log_phase/Log_rep5_plus.wig.gz'
    assert input_filepaths['log']['plus']['rep_5'].is_file()
    
    #### Minus strand
    input_filepaths['log']['minus']['rep_1'] = processed_data_folderpath / 'Log_phase/Log_rep1_minus.wig.gz'
    assert input_filepaths['log']['minus']['rep_1'].is_file()

    input_filepaths['log']['minus']['rep_2'] = processed_data_folderpath / 'Log_phase/Log_rep2_minus.wig.gz'
    assert input_filepaths['log']['minus']['rep_2'].is_file()
    
    input_filepaths['log']['minus']['rep_3'] = processed_data_folderpath / 'Log_phase/Log_rep3_minus.wig.gz'
    assert input_filepaths['log']['minus']['rep_3'].is_file()
    
    input_filepaths['log']['minus']['rep_4'] = processed_data_folderpath / 'Log_phase/Log_rep4_minus.wig.gz'
    assert input_filepaths['log']['minus']['rep_4'].is_file()
    
    input_filepaths['log']['minus']['rep_5'] = processed_data_folderpath / 'Log_phase/Log_rep5_minus.wig.gz'
    assert input_filepaths['log']['minus']['rep_5'].is_file()


    # ----------------------------------------------------------

    ### Stationary phase
    #### Plus strand
    input_filepaths['stat']['plus']['rep_1'] = processed_data_folderpath / 'Stat_phase/Stat_rep1_plus.wig.gz'
    assert input_filepaths['stat']['plus']['rep_1'].is_file()
    
    input_filepaths['stat']['plus']['rep_2'] = processed_data_folderpath / 'Stat_phase/Stat_rep2_plus.wig.gz'
    assert input_filepaths['stat']['plus']['rep_2'].is_file()
    
    input_filepaths['stat']['plus']['rep_3'] = processed_data_folderpath / 'Stat_phase/Stat_rep3_plus.wig.gz'
    assert input_filepaths['stat']['plus']['rep_3'].is_file()

    #### Minus strand
    input_filepaths['stat']['minus']['rep_1'] = processed_data_folderpath / 'Stat_phase/Stat_rep1_minus.wig.gz'
    assert input_filepaths['stat']['minus']['rep_1'].is_file()
    
    input_filepaths['stat']['minus']['rep_2'] = processed_data_folderpath / 'Stat_phase/Stat_rep2_minus.wig.gz'
    assert input_filepaths['stat']['minus']['rep_2'].is_file()
    
    input_filepaths['stat']['minus']['rep_3'] = processed_data_folderpath / 'Stat_phase/Stat_rep3_minus.wig.gz'
    assert input_filepaths['stat']['minus']['rep_3'].is_file()
    
    
    #=======================================================================================================
    
    # Input SeqEnd GFF files
    GFF_input_filepaths = {}

    ## Log and stationary phases
    GFF_input_filepaths['log'] = {}
    GFF_input_filepaths['stat'] = {}
    
    ## Replicates

    ### Log phase
    
    GFF_input_filepaths['log']['rep_1'] = processed_data_folderpath / 'Log_phase/Log_rep1.gff'
    assert GFF_input_filepaths['log']['rep_1'].is_file()

    GFF_input_filepaths['log']['rep_2'] = processed_data_folderpath / 'Log_phase/Log_rep2.gff'
    assert GFF_input_filepaths['log']['rep_2'].is_file()
    
    GFF_input_filepaths['log']['rep_3'] = processed_data_folderpath / 'Log_phase/Log_rep3.gff'
    assert GFF_input_filepaths['log']['rep_3'].is_file()
    
    GFF_input_filepaths['log']['rep_4'] = processed_data_folderpath / 'Log_phase/Log_rep4.gff'
    assert GFF_input_filepaths['log']['rep_4'].is_file()
    
    GFF_input_filepaths['log']['rep_5'] = processed_data_folderpath / 'Log_phase/Log_rep5.gff'
    assert GFF_input_filepaths['log']['rep_5'].is_file()

    ### Stationary phase
    
    GFF_input_filepaths['stat']['rep_1'] = processed_data_folderpath / 'Stat_phase/Stat_rep1.gff'
    assert GFF_input_filepaths['stat']['rep_1'].is_file()
    
    GFF_input_filepaths['stat']['rep_2'] = processed_data_folderpath / 'Stat_phase/Stat_rep2.gff'
    assert GFF_input_filepaths['stat']['rep_2'].is_file()
    
    GFF_input_filepaths['stat']['rep_3'] = processed_data_folderpath / 'Stat_phase/Stat_rep3.gff'
    assert GFF_input_filepaths['stat']['rep_3'].is_file()
    
    
    # Input suppl table experimental terms
    
    exp_terms_filename = input_folderpath.parents[1] / '2024-02_13_from_BMC_Biol_suppl/Suppl_data_1.xlsx'
    assert exp_terms_filename.is_file()

## Set constants & parameters

### Constants

In [ ]:
# Random seed
seed(3)

# Nucleotides
bases = "acgt"
lett_to_index = dict(zip(bases, range(4)))

# Display options (adjusted to Mac)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Pandas options
pd.set_option('display.max_columns', None)

# Normalise to read coverage?
is_expr_norm = True

### Parameters

In [ ]:
# Normalisation factor while calculating read-through rate: 
# t+rt - terminated & read-through trancripts count, 
# t+rt+ri - terminated & read-through & (re)initiated trancripts count 
norm_factor = 't+rt' # 't+rt+ri'

# Whether or not to delete efficiency = 1, 0, <0
is_eff_1_deleted = True
is_eff_0_deleted = True
is_neg_eff_deleted = True

# Efficiency tresholds
max_eff_th = 1000

# Expression tresholds
min_expr_th = 0.1
max_expr_th = 15

# Length of fragment before and after terminator

before_term_len = 130
before_exclude_len = 60
after_gap = 10 # space to ignore after term coord
after_term_len = 80

before_region_suffix = '_before_reg('+str(-before_term_len)+','+str(-before_exclude_len)+')'

# Efficiency (wig files) or readthrough/reinitiation/termination (gff files)
eff_measure = 'readthrough'#'reinitiation'#'termination'


## Install and import in-house functions

### Special functions

In [ ]:
def merge_between_full(df_value, df_interval, value_col, interval_cols, closed, data_type):
    # Inner join by interval (SQL between analog)
    
    ## df_interval[interval_cols[0]] <(=) df_value[value] <(=) df_interval[interval_cols[1]]
    ## closed: both - [a,b]; right - (a,b]; left - [a,b); neither - (a,b)
    ## data_type: int64 for int or smth else for float
    
    # Drop NAs and inf
    df_interval[interval_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
    df_interval = df_interval.dropna(subset=interval_cols)
    df_value.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_value = df_value.dropna(subset=[value_col])
    
    # Unify data types
    df_interval.loc[:,interval_cols] = df_interval.loc[:,interval_cols].astype(data_type)
    df_value.loc[:,value_col] = df_value.loc[:,value_col].astype(data_type)

    
    a = df_value[value_col].values
    bl = df_interval[interval_cols[0]].values
    bh = df_interval[interval_cols[1]].values

    if closed == 'both': # [bl,bh]
        i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))
    elif closed == 'right': # (bl,bh]
        i, j = np.where((a[:, None] > bl) & (a[:, None] <= bh))
    elif closed == 'left': # [bl,bh)
        i, j = np.where((a[:, None] >= bl) & (a[:, None] < bh))
    elif closed == 'neither': # (bl,bh)
        i, j = np.where((a[:, None] > bl) & (a[:, None] < bh))
    else:
        raise TypeError('Uncknown type of interaval: '+str(closed))
    

    merged_df = pd.DataFrame(
        np.column_stack([df_value.values[i], df_interval.values[j]]),
        columns=df_value.columns.append(df_interval.columns)
    )
    
    return merged_df


# Count trancripts +- 10 nt around TTS from wig files
def before_after_counts(coords, file_to_open, strand, before_term_len, after_term_len, before_exclude_len): # to change
    # Coord of TTS is defined as the last base of the transcript
    # after_gap -  intermidiate nts around TSS were excluded from counting
    coords.sort()
    coords_len = len(coords)
    before_cnts = []
    after_cnts = []
    i = 0

    with gzip.open(file_to_open,'r') as fin:
        if strand == 'plus':
            before_cnt = 0
            after_cnt = 0
            curr_line_num = -1
            #print('i = '+ str(i)+', coord = '+ str(coords[i]))
            for line in fin:
                curr_coord = coords[i]
                #print(str(curr_line_num)+ ' '+ str(line.strip()))
                if (curr_line_num >= curr_coord - before_term_len) and (curr_line_num < curr_coord - before_exclude_len): 
                    before_cnt += int(line.strip())
                elif (curr_line_num > curr_coord + after_gap) and (curr_line_num <= curr_coord + after_term_len + after_gap): # exclude 2 intermidiate nt
                    after_cnt += int(line.strip())
                    if curr_line_num == curr_coord + after_term_len + after_gap:
                        before_cnts.append(before_cnt/before_term_len)
                        after_cnts.append(after_cnt/after_term_len)
                        i += 1
                        if i < coords_len: 
                            #print('i = '+ str(i)+', coord = '+ str(coords[i]))
                            before_cnt = 0
                            after_cnt = 0
                        else:
                            before_cnt = 0
                            after_cnt = 0
                            i = 0
                            break
                            
                curr_line_num += 1
                
        elif strand == 'minus':
            before_cnt = 0
            after_cnt = 0
            curr_line_num = -1
            #print('i = '+ str(i)+', coord = '+ str(coords[i]))
            for line in fin:
                curr_coord = coords[i]
                #print(str(curr_line_num)+ ' '+ str(line.strip))
                if (curr_line_num >= curr_coord - after_term_len - after_gap) and (curr_line_num < curr_coord - after_gap): # exclude 3 intermidiate nt
                    after_cnt += int(line.strip())
                elif (curr_line_num > curr_coord + before_exclude_len) and (curr_line_num <= curr_coord + before_term_len): # exclude 3 intermidiate nt
                    before_cnt += int(line.strip())
                    if curr_line_num == curr_coord + before_term_len - 1:
                        before_cnts.append(before_cnt/before_term_len)
                        after_cnts.append(after_cnt/after_term_len)
                        i += 1
                        if i < coords_len: 
                            #print('i = '+ str(i)+', coord = '+ str(coords[i]))
                            before_cnt = 0
                            after_cnt = 0
                        else:
                            before_cnt = 0
                            after_cnt = 0
                            i = 0
                            break
                            
                curr_line_num += 1
    
    return coords, before_cnts, after_cnts

# Preprocessing

Import terminator data, filter & modify the data

## Import data

### Import terminator files (with coords and props)

In [ ]:
# Import terminators dataframe
in_terms_df = pd.read_excel(input_xls_terms_filename, sheet_name='Sheet1',header=1, names=None, index_col=0, dtype=None, skiprows=0)
## Rename some columns
in_terms_df = in_terms_df.rename(columns={" ∆G (kcal/mol)": "dG", "5'_flank_sequence": "5_prime_seq", "Number_of_A_around_5'flank":"Number_of_A_around_5_prime", "3'_flank_sequence":"3_prime_seq", "Number_of_U_around_5'flank":"Number_of_U_around_5_prime", "%GC_of_stem_region(left)":"GC_content_of_stem_region(left)" })

print('in_terms_df:')
display(in_terms_df)

# Import bidir terminators dataframe
in_bidir_terms_df = pd.read_excel(input_xls_bidir_terms_filename, sheet_name='Extended Data Table 5',header=1, names=None, index_col=0, dtype=None, skiprows=0)
## Rename some columns
in_bidir_terms_df = in_bidir_terms_df.rename(columns={"Overlap_start_site(5'_of _plus_strand)": "Overlap_start_site", "Overlap_end_site(5'_of _plus_strand)":"Overlap_end_site", " ∆G (kcal/mol)":"dG", "5'_flank_of_hairpin_structure_around_overlapping_region":"5_prime_of_hairpin_structure_around_overlapping_region", "Number_of_nucleotide_A_of_5'_flank":"Number_of_nucleotide_A_of_5_prime", "3'_flank_of_hairpin_structure_around_overlapping_region":"3_prime_of_hairpin_structure_around_overlapping_region", "Number_of_nucleotide_U_of_3'flank":'Number_of_nucleotide_U_of_3_prime', "GC%_of_stem_RNA_sequence":"GC_content_of_stem_RNA_sequence"})

print('in_bidir_terms_df:')
display(in_bidir_terms_df)


## Filter and modily data

### Add column is_bidir to terms_df, fill NAs, export full dataset to Excel

In [ ]:
# Interval-based join between bidir_terms_df and terms_df
## Add is_bidir to bidir_terms_df
in_bidir_terms_df['is_bidir'] = True
## Join
bidir_terms_df = merge_between_full(in_terms_df, in_bidir_terms_df, "TTS_position", ["Overlap_start_site","Overlap_end_site"], "both", "int64")

# Left join between in_terms df and bidir_terms_df to include is_bidir colums
## Make cute version of bidir_terms_df
cute_bidir_terms_df = bidir_terms_df[['TTS_position', 'TTS_strand','is_bidir']]
# Left join
terms_df = pd.merge(in_terms_df, cute_bidir_terms_df, how='left', on=['TTS_position','TTS_strand'])
## Delete clutter and fill NA
terms_df['is_bidir'].replace([np.nan], False, inplace=True)
terms_df['intrinsic_TTS'].replace([np.nan], False, inplace=True)
terms_df['intrinsic_TTS'].replace('intrinsic_TTS', True, inplace=True)
terms_df['TTS_type_rho_dependent'].replace([np.nan], False, inplace=True)
terms_df['TTS_type_rho_dependent'].replace('rho-dependent', True, inplace=True)

# Add unique TTS_id
terms_df['TTS_id'] = terms_df['TTS_position'].astype(str) +'_'+ terms_df['TTS_strand'].astype(str)
## Make TTS_id the first column
terms_df.insert(0, 'TTS_id', terms_df.pop('TTS_id'))
print('terms_df:')
display(terms_df)

# Export full Ju et all dataset to Excel
terms_df.to_excel(Ju_full_excel_table)


### Only intrinsic (xor Rho-dependent)  unidirectional (xor bidirectional) terminators 

In [ ]:
if term_mechanism == 'Rho':
    if is_bidir:
        final_terms_df = terms_df[~terms_df['intrinsic_TTS'] & terms_df['is_bidir'] & terms_df['TTS_type_rho_dependent']]
    else:
        final_terms_df = terms_df[~terms_df['intrinsic_TTS'] & ~terms_df['is_bidir'] & terms_df['TTS_type_rho_dependent']]
elif term_mechanism == 'intrinsic':
    if is_bidir:
        final_terms_df = terms_df[terms_df['intrinsic_TTS'] & (terms_df['is_bidir']) & ~(terms_df['TTS_type_rho_dependent'])]
    else:
        final_terms_df = terms_df[terms_df['intrinsic_TTS'] & ~(terms_df['is_bidir']) & ~(terms_df['TTS_type_rho_dependent'])]

display(final_terms_df)

cute_final_terms_df = final_terms_df[['TTS_position','TTS_strand']]
display(cute_final_terms_df)

# Body

## Term efficiency (wig files-based)

### Loop over wig files

Loop over all input files, open each file, run counting function

In [ ]:
for phase in phases:
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
     
    for strand in strands:
        curr_terms_df = cute_final_terms_df[cute_final_terms_df['TTS_strand']==sign_strand[strand]]
        effs = []
        before_exprs = []
        for rep in reps:
            file_to_open = input_filepaths[phase][strand][rep]
            print('the file opened: '+ str(file_to_open))
            curr_colname_suffix = phase+'_'+rep
            
            ## Run counting script
            coords = curr_terms_df['TTS_position'].tolist()
            coords, before_cnts, after_cnts = before_after_counts(coords, file_to_open, strand, before_term_len, after_term_len, before_exclude_len)
            eff = []
            for i, j in zip(after_cnts, before_cnts):
                if j != 0:
                    eff.append((j - i) / j)
                else:
                    eff.append(np.nan)

            effs.append(eff)
            
            curr_cnt_terms_df = pd.DataFrame(list(zip(coords, before_cnts, after_cnts, eff)), columns =['TTS_position', curr_colname_suffix+'_before_cnt', curr_colname_suffix+'_after_cnt', curr_colname_suffix+'_eff'])
            curr_cnt_terms_df['TTS_strand'] = sign_strand[strand]
            
  
            
            ### Inner join curr_cnt_terms_df to curr_terms_df for each strand separately
            curr_terms_df = pd.merge(curr_terms_df, curr_cnt_terms_df, how='inner', on=['TTS_position','TTS_strand'])
        
        if strand == 'plus':
            plus_curr_terms_df = copy.deepcopy(curr_terms_df)
        elif strand == 'minus':
            minus_curr_terms_df = copy.deepcopy(curr_terms_df)
    
    plus_minus_curr_term_df = pd.concat([plus_curr_terms_df, minus_curr_terms_df])
    
    # Left join counts to final_terms_df
    final_terms_df = pd.merge(final_terms_df, plus_minus_curr_term_df, how='left', on=['TTS_position','TTS_strand'])
          

### Normalise before_counts to total coverage
* Normilise each repicate
* Add avareges over normalised counts across replicates

In [ ]:
# Normilise each repicate
for phase in phases:
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
    
    for rep in reps:
        curr_colname_suffix = phase+'_'+rep
        curr_before_cnt_colname = curr_colname_suffix+'_before_cnt'
        curr_after_cnt_colname = curr_colname_suffix+'_after_cnt'
        norm_expr_colname = curr_colname_suffix+'_norm_expr'
        #print(curr_before_cnt_colname)
        
        ## Normalise expressions
        curr_before_cnts = final_terms_df[curr_before_cnt_colname].tolist()
        avg_expr = sum(curr_before_cnts)/len(curr_before_cnts)
        #print(avg_expr)
        norm_exprs = [val/avg_expr for val in curr_before_cnts]
       
        ## Insert into databse after curr_after_cnt_colname
        loc  = final_terms_df.columns.get_loc(curr_after_cnt_colname) + 1
        final_terms_df.insert(loc, norm_expr_colname, norm_exprs)

# Add avareges over normalised counts across replicates
for phase in phases:
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
    
    # Collect norm expr & eff colnames to average
    norm_expr_colnames = []
    eff_colnames = []
    for rep in reps:
        curr_colname_suffix = phase+'_'+rep
        
        curr_norm_expr_colname = curr_colname_suffix+'_norm_expr'
        norm_expr_colnames.append(curr_norm_expr_colname)
        
        curr_eff_colname = curr_colname_suffix+'_eff'
        eff_colnames.append(curr_eff_colname)
    
    phase_avg_expr_colname = phase+'_avg_norm_expr'
    final_terms_df[phase_avg_expr_colname] = final_terms_df[norm_expr_colnames].mean(axis='columns')
    
    phase_avg_eff_colname = phase+'_avg_eff'
    final_terms_df[phase_avg_eff_colname] = final_terms_df[eff_colnames].mean(axis='columns')
       

## Readthrough vs reinitiation (GFF trancripts-based)

### Count terminated, read-through, reinitiated trancripts (function)

In [ ]:
def transcript_term_cnt(transcript_df, term_coord, strand, norm_factor):
# Count 3 numbers for each terminator: terminated trancripts, read-through transcripts, reinitiated trancripts 
# norm_factor: what to normilize to when calculating rt, ri percentages: 't+rt+ri' or 't+rt'

# NOTE:
# Terminator windows start/stop are in line with transcription direction (before_start, before_end, after_start, after_end)
# Transcript start/stop are in line with reference genome direction (transcripts.df['start'], transcripts.df['end'])

    if strand == '+':
        before_start = term_coord - before_term_len
        before_end = term_coord - before_exclude_len
        #print('before_start = '+str(before_start))
        #print('before_end = '+str(before_end))

        #print('Before window overlap:')
        before_overlap = transcript_df[(transcript_df['start']<before_end) & (transcript_df['end']>=before_start) & (transcript_df['strand']==strand)]
        #display(before_overlap)

        after_start = term_coord + after_gap
        after_end = term_coord + after_term_len + after_gap
        #print('after_start = '+str(after_start))
        #print('after_end = '+str(after_end))

        #print('After window overlap:')
        after_overlap = transcript_df[(transcript_df['start']<=after_end) & (transcript_df['end']>after_start) & (transcript_df['strand']==strand)]
        #display(after_overlap)

    elif strand == '-':
        before_start = term_coord+before_term_len
        before_end = term_coord+before_exclude_len
        #print('before_start = '+str(before_start))
        #print('before_end = '+str(before_end))

        #print('Before window overlap:')
        before_overlap = transcript_df[(transcript_df['start']<=before_start) & (transcript_df['end']>before_end) & (transcript_df['strand']==strand)]
        #display(before_overlap)

        after_start = term_coord - after_gap
        after_end = term_coord - after_term_len - after_gap
        #print('after_start = '+str(after_start))
        #print('after_end = '+str(after_end))

        #print('After window overlap:')
        after_overlap = transcript_df[(transcript_df['start']<after_start) & (transcript_df['end']>=after_end) & (transcript_df['strand']==strand)]
        #display(after_overlap)

    else:
        raise TypeError('What a strand is it?')

    before_tr_ids = set(before_overlap['transcript_id'].tolist())
    #print(before_tr_ids)

    after_tr_ids = set(after_overlap['transcript_id'].tolist())
    #print(after_tr_ids)

    # Terminated
    Terminated = before_tr_ids.difference(after_tr_ids)
    Terminated_cnt = len(Terminated)
    #print('Terminated:')
    #print(Terminated_cnt)

    # Read-through
    Readthrough = before_tr_ids.intersection(after_tr_ids)
    Readthrough_cnt = len(Readthrough)
    #print('Readthrough:')
    #print(Readthrough_cnt)

    # Reinitiated
    Reinitiated = after_tr_ids.difference(before_tr_ids)
    Reinitiated_cnt = len(Reinitiated)
    #print('Reinitiated:')
    #print(Reinitiated_cnt)
    
    # Percentages
    if norm_factor == 't+rt+ri':
        
        All_cnt = Terminated_cnt + Readthrough_cnt + Reinitiated_cnt
        if All_cnt == 0:
            Terminated_pc = np.nan
            Readthrough_pc = np.nan
            Reinitiated_pc = np.nan

        else:
            Terminated_pc = Terminated_cnt / All_cnt
            Readthrough_pc = Readthrough_cnt / All_cnt
            Reinitiated_pc = 1 - Terminated_pc - Readthrough_pc
            
    elif norm_factor == 't+rt':
        
        All_cnt = Terminated_cnt + Readthrough_cnt
        if All_cnt == 0:
            Terminated_pc = np.nan
            Readthrough_pc = np.nan
            Reinitiated_pc = np.nan

        else:
            Terminated_pc = Terminated_cnt / All_cnt
            Readthrough_pc = Readthrough_cnt / All_cnt
            Reinitiated_pc = Reinitiated_cnt / All_cnt
    else:
        raise TypeError('Unknown norm_factor: '+ str(norm_factor))

            
    
    return Terminated_cnt, Readthrough_cnt, Reinitiated_cnt, Terminated_pc, Readthrough_pc, Reinitiated_pc


### Count t, rt, ri trancripts for each term, replicate -> insert into final_terms_df

In [ ]:
for phase in phases:
    
    print('phase = '+phase)
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
        
    for rep in reps:
        
        print('rep = '+rep)
                
        # Import SEnd mapped trancripts (GFF files)
        transcripts = gffpd.read_gff3(GFF_input_filepaths[phase][rep])
        #print(transcripts.header)
        #display(transcripts.df)

        # Unique transcript_id
        transcripts.df['transcript_id'] = transcripts.df.index
        cols = transcripts.df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        transcripts.df = transcripts.df[cols]
    
        # Counts
        final_terms_df[phase+'_'+rep+'_t'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[0], axis=1)
        final_terms_df[phase+'_'+rep+'_rt'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[1], axis=1)
        final_terms_df[phase+'_'+rep+'_ri'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[2], axis=1)
        
        # Persentages
        final_terms_df[phase+'_'+rep+'_pc_t'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[3], axis=1)
        final_terms_df[phase+'_'+rep+'_pc_rt'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[4], axis=1)
        final_terms_df[phase+'_'+rep+'_pc_ri'] = final_terms_df.apply(lambda row: transcript_term_cnt(transcripts.df, row['TTS_position'], row['TTS_strand'], norm_factor)[5], axis=1)

    # Add phase averages
    def avg_eff_measure(row, reps, phase, measure_suffix):
        # measure_suffix: 'pc_t', 'pc_rt', 'pc_ri'
        summa = 0
        for rep in reps:
            summa += row[phase+'_'+rep+'_'+measure_suffix]
            
        return summa/len(reps)
            
    ## Average termination    
    measure_suffix = 'pc_t'
    final_terms_df[phase+'_t_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)
    
    ## Average readthrough    
    measure_suffix = 'pc_rt'
    final_terms_df[phase+'_rt_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)
        
    ## Average reinitiation    
    measure_suffix = 'pc_ri'
    final_terms_df[phase+'_ri_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)


### Normalize trancript counts: avg_t+rt for each rep, phase; norm function

In [ ]:
# Rep average of upstream counts (for normalization)
avg_before_cnt = {}

for phase in phases:
    avg_before_cnt[phase] = {}
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
        
    for rep in reps:
        avg_before_cnt[phase][rep] = {}
        
        curr_rep_counts = {} # Counts for for the current replicete (for averaging)
        curr_rep_counts['t'] = final_terms_df[phase+'_'+rep+'_t'].tolist() # terminated
        curr_rep_counts['rt'] = final_terms_df[phase+'_'+rep+'_rt'].tolist() # read_through
        
        avg_before_cnt[phase][rep] = (sum(curr_rep_counts['t']) + sum(curr_rep_counts['rt']))/len(curr_rep_counts['t'])
        
        
# Normalization function
def norm_tr_cnt(curr_tr_cnt, avg_before_cnt, phase, rep): 
    return curr_tr_cnt/avg_before_cnt[phase][rep]


# Add normilized t, rt, ri counts to final terms_df
for phase in phases:

    print('phase = '+phase)
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
        
    for rep in reps:
        
        print('rep = '+rep)
    
        # Normilized counts
        final_terms_df[phase+'_'+rep+'_t_norm'] = final_terms_df.apply(lambda row: norm_tr_cnt(row[phase+'_'+rep+'_t'], avg_before_cnt, phase, rep), axis=1)
        final_terms_df[phase+'_'+rep+'_rt_norm'] = final_terms_df.apply(lambda row: norm_tr_cnt(row[phase+'_'+rep+'_rt'], avg_before_cnt, phase, rep), axis=1)
        final_terms_df[phase+'_'+rep+'_ri_norm'] = final_terms_df.apply(lambda row: norm_tr_cnt(row[phase+'_'+rep+'_ri'], avg_before_cnt, phase, rep), axis=1)
        


    # Add phase averages
    def avg_eff_measure(row, reps, phase, measure_suffix):
        # measure_suffix: '_t_norm', '_rt_norm', '_ri_norm'
        summa = 0
        for rep in reps:
            summa += row[phase+'_'+rep+'_'+measure_suffix]
            
        return summa/len(reps)
            
    ## Average norm termination    
    measure_suffix = 't_norm'
    final_terms_df[phase+'_t_norm_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)
    
    ## Average norm readthrough    
    measure_suffix = 'rt_norm'
    final_terms_df[phase+'_rt_norm_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)
        
    ## Average norm reinitiation    
    measure_suffix = 'ri_norm'
    final_terms_df[phase+'_ri_norm_avg'] = final_terms_df.apply(lambda row:  avg_eff_measure(row, reps, phase, measure_suffix), axis=1)

### Add experimntal terms to final_terms_df

In [ ]:
# Import terminators dataframe
exp_terms_df = pd.read_excel(exp_terms_filename, sheet_name='Ju et al. overlap',header=1, names=None, index_col=0, dtype=None, skiprows=0)

# Drop unnecesarry columns
exp_terms_df = exp_terms_df[['New Name', 'Ju_IT_ID', 'Strands: exp/Ju', 'Comments']]

# Rename some columns
exp_terms_df = exp_terms_df.rename(columns={'New Name': 'Exp_term_name', 'Ju_IT_ID': 'TTS_id'})
print('exp_terms_df:')
display(exp_terms_df)

# Marge with final_terms_df

final_terms_df = pd.merge(final_terms_df, exp_terms_df, how='left', on='TTS_id')

## Graphs and scatterplots

### Show and save the final dataframe 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(final_terms_df)
    
final_terms_df.to_excel(final_excel_table)


### Eff measure vs expression: each phase & replicate

In [ ]:
if_color_str_feature = False
str_colname = 'Number_of_A_around_5_prime'
scale_factor = 1
if if_color_str_feature:
    print('Dot colour is '+str_colname)

rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

expr_colnames = []
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')        
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
   
            
           
fig_s, ax_s = plt.subplots(rep_number,2)
fig_s.set_size_inches(7*2,7*rep_number)
fig_s.suptitle('Terminator '+eff_measure+' vs. expression: '+ experiment_set+', each phase & replicate seperately')

for i in range(rep_number):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df[[expr_colname, eff_colname, str_colname]].dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        
    expressions = curr_final_terms_df[expr_colname].tolist()
    efficiencies = curr_final_terms_df[eff_colname].tolist()
    STRs = curr_final_terms_df[str_colname].tolist() # structural feature
    #print(np.array(STRs)*scale_factor)
    
    if is_expr_norm:
        
        ax_s[i,0].set_title(eff_colname, fontsize=10)
        
        if if_color_str_feature:
            pc = ax_s[i,0].scatter(expressions, efficiencies, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,0].scatter(expressions, efficiencies, s=4)
        

        ax_s[i,0].set_xlabel('Expression')
        ax_s[i,0].set_ylabel('Terminator '+eff_measure+ ' rate')

        # Correlations
        r, p = scipy.stats.pearsonr(expressions, efficiencies)
        rho, prho = scipy.stats.spearmanr(expressions, efficiencies)
        tau, ptau = scipy.stats.kendalltau(expressions, efficiencies)
        #ax_s[i,0].annotate('    Person R = '+ str(r.round(1)) + ', p = '+ str(p.round(2))+ '\n    Spearman rho =  '+ str(rho.round(1))+ ', p = '+ str(prho.round(4))+ '\n    Kendall tau =  '+ str(tau.round(1))+ ', p = '+ str(ptau.round(4)), (1, 0))
        if (eff_measure == 'efficiency') or (eff_measure == 'termination'):
            ax_s[i,0].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (0.2, 0))
        else:
            ax_s[i,0].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (7, 0.8))
       

        ax_s[i,0].set_xlim(-0.5, max_expr_th+0.5)
        ax_s[i,0].set_ylim(-0.05, 1.05)


        #ax_s[i,0].set_xticks(np.arange(0, max_expr_th, step=0.5))
        #ax_s[i,0].set_xticklabels(np.arange(0, max_expr_th, step=0.5))

        ax_s[i,0].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,0].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))


        ax_s[i,1].set_title(eff_colname, fontsize=10)
        ax_s[i,1].set_xscale('log')

        #ax_s[i,1].scatter(expressions, efficiencies, s=6)
        if if_color_str_feature:
            pc = ax_s[i,1].scatter(expressions, efficiencies, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,1].scatter(expressions, efficiencies, s=4)

        ax_s[i,1].set_xlabel('Expression')
        ax_s[i,1].set_ylabel('Terminator '+eff_measure+ ' rate')

        #ax_s[i,1].set_xlim(1, 100000)
        ax_s[i,1].set_ylim(-0.05, 1.05)


        #ax_s[i,1].set_xticks(np.arange(0, 10000, step=1000))
        #ax_s[i,1].set_xticklabels(np.arange(0, 10000, step=1000))

        ax_s[i,1].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,1].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))
        
        # Colorbar
        if if_color_str_feature:
            cax = ax_s[i,1].inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax_s[i,1].transAxes)
            colorbar = fig_s.colorbar(pc, cax=cax)
            colorbar.set_label(str_colname, rotation=270, va="baseline")


fig_s.show()
fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_rates_reps_phases.pdf', bbox_inches='tight')
       

### Upstream expression vs downstraem expression: each phase & replicate 

In [ ]:
is_dn_exp_wig = True # True - expression is mease on wig files as in  Eff measure vs expression: each phase & replicate
# Make sense only for rt and ri
if eff_measure == 'readthrough' or eff_measure == 'reinitiation':
    if_color_str_feature = False
    str_colname = 'Number_of_A_around_5_prime'
    scale_factor = 1
    if if_color_str_feature:
        print('Dot colour is '+str_colname)

    rep_number = len(log_reps) + len(stat_reps)
    all_reps_phases = log_reps + stat_reps

    up_expr_colnames = []
    dn_expr_colnames = []
    
    expr_colnames = []

    for phase in phases:
        if phase == 'log':
            for rep in log_reps:

                up_expr_colnames.append([phase+'_'+rep+'_t_norm', phase+'_'+rep+'_rt_norm'])        

                if eff_measure == 'readthrough':
                    dn_expr_colnames.append(phase+'_'+rep+'_rt_norm')
                elif eff_measure == 'reinitiation':
                    dn_expr_colnames.append(phase+'_'+rep+'_ri_norm')
                else: 
                    raise TypeError('Unknown eff measure: '+str(eff_measure))
                    
                expr_colnames.append('log_'+rep+'_norm_expr')

        elif phase == 'stat':
            for rep in stat_reps:

                up_expr_colnames.append([phase+'_'+rep+'_t_norm', phase+'_'+rep+'_rt_norm'])        

                if eff_measure == 'readthrough':
                    dn_expr_colnames.append(phase+'_'+rep+'_rt_norm')
                elif eff_measure == 'reinitiation':
                    dn_expr_colnames.append(phase+'_'+rep+'_ri_norm')
                else: 
                    raise TypeError('Unknown eff measure: '+str(eff_measure))
                    
                expr_colnames.append('stat_'+rep+'_norm_expr')



    fig_s, ax_s = plt.subplots(rep_number,2)
    fig_s.set_size_inches(7*2,7*rep_number)
    fig_s.suptitle('Terminator upstream expr (t+rt) vs.downstream expression ('+eff_measure+'): '+ experiment_set+', each phase & replicate seperately')

    for i in range(rep_number):

        up_expr_colname = up_expr_colnames[i]
        dn_expr_colname = dn_expr_colnames[i]
        
        expr_colname = expr_colnames[i]
        
        if is_dn_exp_wig: # old style expression
            up_expr_colname = expr_colname
            all_colnames = list(set([up_expr_colname] + [dn_expr_colname] + [str_colname]))
        else:
            all_colnames = list(set(up_expr_colname + [dn_expr_colname] + [str_colname]))
            
        #print(all_colnames)
        # Drop NA values (and ones?)
        if is_dn_exp_wig: # old style expression
            curr_final_terms_df = final_terms_df[all_colnames].dropna(how='any', subset=[up_expr_colname, dn_expr_colname])
        else:
            curr_final_terms_df = final_terms_df[all_colnames].dropna(how='any', subset=[up_expr_colname[0], dn_expr_colname])

        # Delelte dn_expr = 1?
        if is_eff_1_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[dn_expr_colname]!=1]

        # Delelte dn_expr = 0?
        if is_eff_0_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[dn_expr_colname]!=0]

        # Delete negative dn_expr?
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[dn_expr_colname] >= 0]

        # Delete dn_expr > max_eff_th
        #curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[dn_expr_colname] <= max_eff_th]


        # Delete expressions < treshold
        if is_neg_eff_deleted:
            if is_dn_exp_wig: # old style expression
                curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[up_expr_colname] >= min_expr_th]
                curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[up_expr_colname] <= max_expr_th]
            else:
                curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[up_expr_colname[0]]+curr_final_terms_df[up_expr_colname[1]] >= min_expr_th]
                curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[up_expr_colname[0]]+curr_final_terms_df[up_expr_colname[1]] <= max_expr_th]

        up_expressions = []
        if  is_dn_exp_wig: # old style expression
            up_expressions = curr_final_terms_df[up_expr_colname].tolist()
        else:
            up_expressions_t = curr_final_terms_df[up_expr_colname[0]].tolist()
            up_expressions_rt = curr_final_terms_df[up_expr_colname[1]].tolist()
            assert len(up_expressions_t) == len(up_expressions_rt)

            for u_e in range(len(up_expressions_t)):
                up_expressions.append(up_expressions_t[u_e]+up_expressions_rt[u_e]) 
        
        dn_expressions = curr_final_terms_df[dn_expr_colname].tolist()
        assert len(up_expressions) == len(dn_expressions)
        STRs = curr_final_terms_df[str_colname].tolist() # structural feature
        #print(np.array(STRs)*scale_factor)



        ax_s[i,0].set_title(dn_expr_colname, fontsize=10)

        if if_color_str_feature:
            pc = ax_s[i,0].scatter(up_expressions, dn_expressions, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,0].scatter(up_expressions, dn_expressions, s=4)


        if is_dn_exp_wig:
            ax_s[i,0].set_xlabel('Upstream expression: wig')
        else:
            ax_s[i,0].set_xlabel('Upstream expression: t+rt')
        ax_s[i,0].set_ylabel('Downstraem expression: '+eff_measure)

        # Correlations
        #print(up_expressions)
        #print(dn_expressions)
        if len(up_expressions)>=2 and len(up_expressions)>=2:
            r, p = scipy.stats.pearsonr(up_expressions, dn_expressions)
            rho, prho = scipy.stats.spearmanr(up_expressions, dn_expressions)
            tau, ptau = scipy.stats.kendalltau(up_expressions, dn_expressions)
        else:
            r = 0
            p = 1
            rho = 0 
            prho = 1
            tau = 0
            ptau = 1

        ax_s[i,0].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (7, 0.8))


        ax_s[i,0].set_xlim(-0.5, max_expr_th+0.5)
        ax_s[i,0].set_ylim(-0.05, 1.05)


        #ax_s[i,0].set_xticks(np.arange(0, max_expr_th, step=0.5))
        #ax_s[i,0].set_xticklabels(np.arange(0, max_expr_th, step=0.5))

        ax_s[i,0].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,0].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))


        ax_s[i,1].set_title(dn_expr_colname, fontsize=10)
        ax_s[i,1].set_xscale('log')

        #ax_s[i,1].scatter(expressions, efficiencies, s=6)
        if if_color_str_feature:
            pc = ax_s[i,1].scatter(up_expressions, dn_expressions, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,1].scatter(up_expressions, dn_expressions, s=4)

        ax_s[i,1].set_xlabel('Up expr: t+rt')
        ax_s[i,1].set_ylabel('Dn expr: '+eff_measure)

        ax_s[i,1].set_xlim(0.1, 20)
        ax_s[i,1].set_ylim(-0.05, 1.05)


        #ax_s[i,1].set_xticks(np.arange(0, 10000, step=1000))
        #ax_s[i,1].set_xticklabels(np.arange(0, 10000, step=1000))

        ax_s[i,1].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,1].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))

        # Colorbar
        if if_color_str_feature:
            cax = ax_s[i,1].inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax_s[i,1].transAxes)
            colorbar = fig_s.colorbar(pc, cax=cax)
            colorbar.set_label(str_colname, rotation=270, va="baseline")



    fig_s.show()
    fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_cnts_reps_phases.pdf', bbox_inches='tight')


### For each phase seperately (replicate averages)

In [ ]:
if_color_str_feature = False
str_colname = 'Number_of_A_around_5_prime'
scale_factor = 1
if if_color_str_feature:
    print('Dot colour is '+str_colname)

for is_counts in [True, False]:# True: Use normalised rt, ri counts intead of proportions(rates)

    expr_colnames = []
    eff_colnames = []


    for phase in phases:
        expr_colnames.append(phase+'_avg_norm_expr')

        if eff_measure == 'efficiency':
            eff_colnames.append(phase+'_avg_eff')
        elif eff_measure == 'readthrough':
            if is_counts:
                eff_colnames.append(phase+'_rt_norm_avg')
            else:
                eff_colnames.append(phase+'_rt_avg')
        elif eff_measure == 'reinitiation':
            if is_counts:
                eff_colnames.append(phase+'_ri_norm_avg')
            else:
                eff_colnames.append(phase+'_ri_avg')
        elif eff_measure == 'termination':
            eff_colnames.append(phase+'_t_avg')
        else: 
            raise TypeError('Unknown eff measure: '+str(eff_measure))


    fig_s, ax_s = plt.subplots(len(phases),2)
    fig_s.set_size_inches(7*2,7*len(phases))
    fig_s.suptitle('Terminator '+eff_measure+' vs. expression: '+experiment_set+', replicate averages')

    for i in range(len(phases)):

        expr_colname = expr_colnames[i]
        eff_colname = eff_colnames[i]
        # Drop NA values (and ones?)
        curr_final_terms_df = final_terms_df[[expr_colname, eff_colname, str_colname]].dropna(how='any', subset=[expr_colname, eff_colname])

        # Delelte eff = 1?
        if is_eff_1_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]

        # Delelte eff = 0?
        if is_eff_0_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]

        # Delete negative efficiencies?
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]

        # Delete efficiencises > max_eff_th
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]

        # Delete expressions < treshold
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]

        expressions = curr_final_terms_df[expr_colname].tolist()
        efficiencies = curr_final_terms_df[eff_colname].tolist()
        STRs = curr_final_terms_df[str_colname].tolist() # structural feature


        ax_s[i,0].set_title(eff_colname, fontsize=10)

        if if_color_str_feature:
            pc = ax_s[i,0].scatter(expressions, efficiencies, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,0].scatter(expressions, efficiencies, s=4)

        ax_s[i,0].set_xlabel('Expression')
        ax_s[i,0].set_ylabel('Terminator '+eff_measure+' rate')

        # Correlations
        r, p = scipy.stats.pearsonr(expressions, efficiencies)
        rho, prho = scipy.stats.spearmanr(expressions, efficiencies)
        tau, ptau = scipy.stats.kendalltau(expressions, efficiencies)
        #ax_s[i,0].annotate('    Person R = '+ str(r.round(1)) + ', p = '+ str(p.round(2))+ '\n    Spearman rho =  '+ str(rho.round(1))+ ', p = '+ str(prho.round(4))+ '\n    Kendall tau =  '+ str(tau.round(1))+ ', p = '+ str(ptau.round(4)), (1, 0))
        if (eff_measure == 'efficiency') or (eff_measure == 'termination'):
            ax_s[i,0].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (5, 0))
        else:
            ax_s[i,0].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (7, 0.8))



        ax_s[i,0].set_xlim(-0.5, max_expr_th+0.5)
        ax_s[i,0].set_ylim(-0.05, 1.05)


        #ax_s[i,0].set_xticks(np.arange(0, max_expr_th, step=0.5))
        #ax_s[i,0].set_xticklabels(np.arange(0, max_expr_th, step=0.5))

        ax_s[i,0].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,0].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))


        ax_s[i,1].set_title(eff_colname, fontsize=10)
        ax_s[i,1].set_xscale('log')

        if if_color_str_feature:
            pc = ax_s[i,1].scatter(expressions, efficiencies, s=10, c=np.array(STRs)*scale_factor)
        else:
            pc = ax_s[i,1].scatter(expressions, efficiencies, s=4)

        ax_s[i,1].set_xlabel('Expression')
        ax_s[i,1].set_ylabel('Terminator '+eff_measure+' rate')

        #ax_s[i,1].set_xlim(-0.05, 100000)
        ax_s[i,1].set_ylim(-0.05, 1.05)


        #ax_s[i,1].set_xticks(np.arange(0, 10000, step=1000))
        #ax_s[i,1].set_xticklabels(np.arange(0, 10000, step=1000))

        ax_s[i,1].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_s[i,1].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))

        # Colorbar
        if if_color_str_feature:
            cax = ax_s[i,1].inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax_s[i,1].transAxes)
            colorbar = fig_s.colorbar(pc, cax=cax)
            colorbar.set_label(str_colname, rotation=270, va="baseline")


    fig_s.show()
    if is_counts:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'cnts_phases.pdf', bbox_inches='tight')  
    else:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_rates_phases.pdf', bbox_inches='tight')  


### Changes between phases (replicate averages)

In [ ]:
# Collect avg expression & avg efficiencies data

for is_counts in [True, False]:# True: Use normalised rt, ri counts intead of proportions(rates)
    
    expr_colnames = []
    eff_colnames = []

    for phase in phases:
        expr_colnames.append(phase+'_avg_norm_expr')

        if eff_measure == 'efficiency':
            eff_colnames.append(phase+'_avg_eff')
        elif eff_measure == 'readthrough':
            if is_counts:
                eff_colnames.append(phase+'_rt_norm_avg')
            else:
                eff_colnames.append(phase+'_rt_avg')
        elif eff_measure == 'reinitiation':
            if is_counts:
                eff_colnames.append(phase+'_ri_norm_avg')
            else:
                eff_colnames.append(phase+'_ri_avg')
        elif eff_measure == 'termination':
            eff_colnames.append(phase+'_t_avg')
        else: 
            raise TypeError('Unknown eff measure: '+str(eff_measure))

    # To calculate differences later 
    all_expressions = []
    all_efficiencies = []

    # Filter final_terms_df
    ## Drop NA values

    # Dot size parameters
    is_dot_size_expression = True
    scale_factor = 100
    str_colname = 'dG'
    curr_final_terms_df = final_terms_df[expr_colnames + eff_colnames +[str_colname]].dropna(how='any')

    ## Apply other filters 
    for i in range(len(phases)):

        expr_colname = expr_colnames[i]
        eff_colname = eff_colnames[i]

        ### Delelte eff = 1?
        if is_eff_1_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]

        ### Delelte eff = 0?
        if is_eff_0_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]

        ### Delete negative efficiencies?
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]

        ### Delete efficiencises > max_eff_th
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]

        ### Delete expressions < treshold
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]

    STRs = curr_final_terms_df[str_colname].tolist() # structural feature
    print('Dot size is '+str_colname)        

    for i in range(len(phases)):

        expr_colname = expr_colnames[i]
        eff_colname = eff_colnames[i]

        expressions = curr_final_terms_df[expr_colname].tolist()
        efficiencies = curr_final_terms_df[eff_colname].tolist()

        all_expressions.append(expressions)
        all_efficiencies.append(efficiencies)

    #============================================================================================    
    # Plot avg expression in stat phase vs avg expr in log phase

    ## Draw scatterplots

    fig_e, ax_e = plt.subplots(1,2)
    fig_e.set_size_inches(7*2,7*1)
    fig_e.suptitle('Avg expression between phases: '+experiment_set+', replicate averages')

    if is_expr_norm:
        ax_e[0].set_title('Expressions between 0 and 1', fontsize=10)

        ax_e[0].scatter(log_expressions, stat_expressions, s=4)

        ## Add 1-1 line to scatterplot 
        ax_e[0].plot(np.array(log_expressions), np.array(log_expressions))

        ax_e[0].set_xlabel('Log phase expression')
        ax_e[0].set_ylabel('Stat phase expression')


        ax_e[0].set_xlim(-0.1, 1.1)
        ax_e[0].set_ylim(-0.1, 1.1)


        ax_e[0].set_xticks(np.arange(0, 1.1, step=0.1))
        ax_e[0].set_xticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))

        ax_e[0].set_yticks(np.arange(0, 1.1, step=0.1))
        ax_e[0].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))


        ax_e[1].set_title('All expressions', fontsize=10)
        #ax_e[1].set_xscale('log')

        ax_e[1].scatter(log_expressions, stat_expressions, s=6)

        ## Add 1-1 line to scatterplot 
        ax_e[1].plot(np.array(log_expressions), np.array(log_expressions))

         # Correlations
        #r, p = scipy.stats.pearsonr(log_expressions, stat_expressions)
        #rho, prho = scipy.stats.spearmanr(log_expressions, stat_expressions)
        #tau, ptau = scipy.stats.kendalltau(log_expressions, stat_expressions)
        #ax_e[0].annotate('    Person R = '+ str(r.round(1)) + ', p = '+ str(p.round(2))+ '\n    Spearman rho =  '+ str(rho.round(1))+ ', p = '+ str(prho.round(4))+ '\n    Kendall tau =  '+ str(tau.round(1))+ ', p = '+ str(ptau.round(4)), (1, 0.7))
        #ax_e[1].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (6, 5))


        ax_e[1].set_xlabel('Log phase expression')
        ax_e[1].set_ylabel('Stat phase expression')

        #ax_e[1].set_xlim(-10000, 10000)
        #ax_e[1].set_ylim(-1.05, 1.05)


        #ax_e[1].set_xticks(np.arange(0, 10000, step=1000))
        #ax_e[1].set_xticklabels(np.arange(0, 10000, step=1000))

        #ax_e[1].set_yticks(np.arange(0, 10, step=1))
        #ax_e[1].set_yticklabels(np.arange(0, 10, step=1).round(decimals=1))

    fig_e.show()
    
    if is_counts:
        fig_e.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_cnts_stat_log_expr.pdf', bbox_inches='tight')
    else:
        fig_e.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_rates_stat_log_expr.pdf', bbox_inches='tight')

    #============================================================================================        

    # Calculate differencies
    ## Efficiency diffs
    eff_with_phases = list(map(list, zip(*all_efficiencies))) # Transpose list of lists
    log_phase_effs = []
    eff_diffs = []

    for phase_eff in eff_with_phases:
        log_phase_effs.append(phase_eff[0])
        eff_dif = phase_eff[1] - phase_eff[0] # stat - log
        eff_diffs.append(eff_dif)

    ## Expression diffs
    expr_with_phases = list(map(list, zip(*all_expressions))) # Transpose list of lists
    log_phase_exprs = []
    expr_diffs = []

    for phase_expr in expr_with_phases:
        log_phase_exprs.append(phase_expr[0])
        expr_dif = phase_expr[1] - phase_expr[0]  # stat - log
        expr_diffs.append(expr_dif)

    # Draw sctterplots

    fig_s, ax_s = plt.subplots(1,2)
    fig_s.set_size_inches(7*2,7*1)
    fig_s.suptitle('Terminator '+eff_measure+' difference vs. expression diff (between phases): '+experiment_set+', replicate averages')

    if is_expr_norm:

        ax_s[0].set_title('Expression diffs between -2.5 and +2.5', fontsize=10)

        if is_dot_size_expression:
            pc = ax_s[0].scatter(expr_diffs, eff_diffs, s=np.array(log_phase_exprs)*scale_factor, c=np.log10(log_phase_effs))
        else:                              
            pc = ax_s[0].scatter(expr_diffs, eff_diffs, s=np.array(STRs)*scale_factor, c=np.log10(log_phase_effs))

        ## Obtain m (slope) and b(intercept) of linear regression line
        m, b = np.polyfit(np.array(expr_diffs), np.array(eff_diffs), 1)

        ## Add linear regression line to scatterplot 
        ax_s[0].plot(np.array(expr_diffs), m*np.array(expr_diffs)+b)

        ax_s[0].set_xlabel('Expression stat - Expression log')
        ax_s[0].set_ylabel(eff_measure+' stat - '+eff_measure+' log')



        ax_s[0].set_xlim(-2.5, 2.5)
        ax_s[0].set_ylim(-1.05, 1.05)


        ax_s[0].set_xticks(np.arange(-2.5, 2.5, step=0.5))
        ax_s[0].set_xticklabels(np.arange(-2.5, 2.5, step=0.5))

        ax_s[0].set_yticks(np.arange(-1, 1.1, step=0.1))
        ax_s[0].set_yticklabels(np.arange(-1, 1.1, step=0.1).round(decimals=1))


        ax_s[1].set_title('All diffs', fontsize=10)
        #ax_s[1].set_xscale('log')

        if is_dot_size_expression:
            scat = ax_s[1].scatter(expr_diffs, eff_diffs, s=np.array(log_phase_exprs)*scale_factor, c=np.log10(log_phase_effs))
        else:
            scat = ax_s[1].scatter(expr_diffs, eff_diffs, s=np.array(STRs)*scale_factor, c=np.log10(log_phase_effs))


        # Dot Size legend
        # produce a legend with a cross-section of sizes from the scatter
        handles, labels = scat.legend_elements(prop="sizes", alpha=0.6)
        ## Only 3 dots on the legend
        mid_0_based_index = round(len(handles)/2)-1
        labels = ['$\mathdefault{1}$', '$\mathdefault{'+str(mid_0_based_index+1)+'}$', '$\mathdefault{'+str(len(handles))+'}$']
        handles = [handles[0],handles[mid_0_based_index],handles[-1]]
        legend2 = ax_s[1].legend(handles, labels, loc="lower left", title="Log phase Expression ",labelspacing=2)

        # Colorbar legend
        cax = ax_s[1].inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax_s[1].transAxes)
        colorbar = fig_s.colorbar(pc, cax=cax)
        colorbar.set_label("log10(Log phase "+eff_measure+")", rotation=270, va="baseline")

        ## Obtain m (slope) and b(intercept) of linear regression line
        m, b = np.polyfit(np.array(expr_diffs), np.array(eff_diffs), 1)

        ## Add linear regression line to scatterplot 
        ax_s[1].plot(np.array(expr_diffs), m*np.array(expr_diffs)+b)

        # Correlations
        r, p = scipy.stats.pearsonr(expr_diffs, eff_diffs)
        rho, prho = scipy.stats.spearmanr(expr_diffs, eff_diffs)
        tau, ptau = scipy.stats.kendalltau(expr_diffs, eff_diffs)
        ax_s[1].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (-2, 0.7))


        ax_s[1].set_xlabel('Expression stat - Expression log')
        ax_s[1].set_ylabel(eff_measure+' stat - '+eff_measure+' log')

        #ax_s[1].set_xlim(-10000, 10000)
        ax_s[1].set_ylim(-1.05, 1.05)


        #ax_s[1].set_xticks(np.arange(0, 10000, step=1000))
        #ax_s[1].set_xticklabels(np.arange(0, 10000, step=1000))

        ax_s[1].set_yticks(np.arange(-1, 1.1, step=0.1))
        ax_s[1].set_yticklabels(np.arange(-1, 1.1, step=0.1).round(decimals=1))

    fig_s.show()
    
    if is_counts:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_cnts_diff.pdf', bbox_inches='tight')
    else:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_rates_diff.pdf', bbox_inches='tight')


### Changes between phases & replicates (all vs all)

In [ ]:
rep_number = len(log_reps) + len(stat_reps)


for is_counts in [True, False]:# True: Use normalised rt, ri counts intead of proportions(rates)
    
    all_reps_phases = []
    expr_colnames = []
    eff_colnames = []

    if is_expr_norm:
        for phase in phases:
            if phase == 'log':
                for rep in log_reps:
                    expr_colnames.append('log_'+rep+'_norm_expr')

                    if eff_measure == 'efficiency':
                        eff_colnames.append('log_'+rep+'_eff')

                    elif eff_measure == 'readthrough':
                        if is_counts:
                            eff_colnames.append('log_'+rep+'_rt_norm')
                        else:
                            eff_colnames.append('log_'+rep+'_pc_rt')

                    elif eff_measure == 'reinitiation':
                        if is_counts:
                            eff_colnames.append('log_'+rep+'_ri_norm')
                        else:
                            eff_colnames.append('log_'+rep+'_pc_ri')

                    elif eff_measure == 'termination':
                        eff_colnames.append('log_'+rep+'_pc_t')
                    else: 
                        raise TypeError('Unknown eff measure: '+str(eff_measure))

                    all_reps_phases.append('log_'+rep)

            elif phase == 'stat':
                for rep in stat_reps:
                    expr_colnames.append('stat_'+rep+'_norm_expr')

                    if eff_measure == 'efficiency':
                        eff_colnames.append('stat_'+rep+'_eff')

                    elif eff_measure == 'readthrough':
                        if is_counts:
                            eff_colnames.append('stat_'+rep+'_ri_norm')
                        else:
                            eff_colnames.append('stat_'+rep+'_pc_ri')

                    elif eff_measure == 'reinitiation':
                        if is_counts:
                            eff_colnames.append('stat_'+rep+'_ri_norm')
                        else:
                            eff_colnames.append('stat_'+rep+'_pc_ri')

                    elif eff_measure == 'termination':
                        eff_colnames.append('stat_'+rep+'_pc_t')
                    else: 
                        raise TypeError('Unknown eff measure: '+str(eff_measure))

                    all_reps_phases.append('stat_'+rep)


    ## Drop NA values
    #curr_final_terms_df = final_terms_df[expr_colnames + eff_colnames].dropna(how='any')  
    curr_final_terms_df = final_terms_df

    all_expressions = []
    all_efficiencies = []
    for i in range(rep_number):

        expr_colname = expr_colnames[i]
        eff_colname = eff_colnames[i]

        expressions = curr_final_terms_df[expr_colname].tolist()
        efficiencies = curr_final_terms_df[eff_colname].tolist()

        all_expressions.append(expressions)
        all_efficiencies.append(efficiencies)

    # Calculate differencies
    ## Efficiency diffs
    eff_with_reps_phases = list(map(list, zip(*all_efficiencies))) # Transpose list of lists
    eff_diffs = []
    log_phase_effs = []

    ## Expression diffs
    expr_with_reps_phases = list(map(list, zip(*all_expressions))) # Transpose list of lists
    expr_diffs = []
    log_phase_exprs = []

    if len(eff_with_reps_phases) != len(expr_with_reps_phases):
        raise TypeError('FIX IT!')


    for i in range(rep_number): # rows
        log_phase_effs.append([])
        eff_diffs.append([])

        log_phase_exprs.append([])
        expr_diffs.append([])

        for j in range(rep_number): # columns
            log_phase_effs[i].append([])
            eff_diffs[i].append([])

            log_phase_exprs[i].append([])
            expr_diffs[i].append([])

            for k in range(len(eff_with_reps_phases)):
                rep_phase_eff = eff_with_reps_phases[k]
                rep_phase_expr = expr_with_reps_phases[k]

                # Filter out NAs
                if (np.isnan(rep_phase_eff[i]) or np.isnan(rep_phase_eff[j]) or np.isnan(rep_phase_expr[i]) or np.isnan(rep_phase_expr[j])):
                    continue
                log_phase_effs[i][j].append(rep_phase_eff[j])
                eff_dif = rep_phase_eff[i] - rep_phase_eff[j] # row - column
                eff_diffs[i][j].append(eff_dif)

                log_phase_exprs[i][j].append(rep_phase_expr[j])
                expr_dif = rep_phase_expr[i] - rep_phase_expr[j] # row - column
                expr_diffs[i][j].append(expr_dif)

    # Draw expression vs expression scatterplots

    fig_e, ax_e = plt.subplots(rep_number,rep_number)
    fig_e.set_size_inches(7*rep_number,7*rep_number)
    fig_e.suptitle('Expression vs. expression: '+ experiment_set+', each phase & replicate seperately')

    for i in range(rep_number): # rows
        for j in range(rep_number): # columns

            if is_expr_norm:

                ax_e[i][j].set_title(all_reps_phases[i]+' vs '+all_reps_phases[j] , fontsize=10)
                
                ax_e[i][j].scatter(all_expressions[i], all_expressions[j], s=4)

                if i != j:
                    ## 1-1 line
                    m, b = np.polyfit(np.array(all_expressions[i]), np.array(all_expressions[i]), 1)

                    ## Add 1-1 line to scatterplot 
                    ax_e[i][j].plot(np.array(all_expressions[i]), m*np.array(all_expressions[i])+b)

                ax_e[i][j].set_xlabel('Expression ' + all_reps_phases[i])
                ax_e[i][j].set_ylabel('Expression ' + all_reps_phases[j])

                # Correlations
                r, p = scipy.stats.pearsonr(all_expressions[i], all_expressions[j])
                rho, prho = scipy.stats.spearmanr(all_expressions[i], all_expressions[j])
                tau, ptau = scipy.stats.kendalltau(all_expressions[i], all_expressions[j])
                ax_e[i][j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (-1, -0.7))

                #ax_e[i][j].set_xlim(-10, 10)
                #ax_e[i][j].set_ylim(-1.05, 1.05)

                #ax_e[i][j].set_xticks(np.arange(-0.5, 20, step=1))
                #ax_e[i][j].set_xticklabels(np.arange(0, 20, step=1))

                #ax_e[i][j].set_yticks(np.arange(-1, 1.1, step=0.1))
                #ax_e[i][j].set_yticklabels(np.arange(-1, 1.1, step=0.1).round(decimals=1))

            else:
                raise TypeError('FIX IT!')


    fig_e.show()
    fig_e.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_expr_vs_expr.pdf', bbox_inches='tight')


    #================================================================================================================    

    # Draw express diff scatterplots

    fig_s, ax_s = plt.subplots(rep_number,rep_number)
    fig_s.set_size_inches(7*rep_number,7*rep_number)
    fig_s.suptitle('Terminator '+eff_measure+' diff  vs. expression diff: '+ experiment_set+', each phase & replicate seperately')

    for i in range(rep_number): # rows
        for j in range(rep_number): # columns

            if is_expr_norm:

                ax_s[i][j].set_title(all_reps_phases[i]+' - '+all_reps_phases[j] , fontsize=10)

                ax_s[i][j].scatter(expr_diffs[i][j], eff_diffs[i][j], s=np.array(log_phase_exprs[i][j])*scale_factor, c=np.log10(log_phase_effs[i][j]))
                #ax_s[1].scatter(expr_diffs, eff_diffs, s=np.array(log_phase_exprs)*scale_factor, c=np.log10(log_phase_effs))
                if j == (rep_number - 1):
                    cax = ax_s[i][j].inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax_s[i][j].transAxes)
                    colorbar = fig_s.colorbar(pc, cax=cax)
                    colorbar.set_label("log10(initial "+eff_measure+")", rotation=270, va="baseline")           
                if i != j:
                    ## Obtain m (slope) and b(intercept) of linear regression line
                    m, b = np.polyfit(np.array(expr_diffs[i][j]), np.array(eff_diffs[i][j]), 1)

                    ## Add linear regression line to scatterplot 
                    ax_s[i][j].plot(np.array(expr_diffs[i][j]), m*np.array(expr_diffs[i][j])+b)

                ax_s[i][j].set_xlabel('Expression diff')
                ax_s[i][j].set_ylabel('Terminator '+eff_measure+' diff')

                # Correlations
                r, p = scipy.stats.pearsonr(expr_diffs[i][j], eff_diffs[i][j])
                rho, prho = scipy.stats.spearmanr(expr_diffs[i][j], eff_diffs[i][j])
                tau, ptau = scipy.stats.kendalltau(expr_diffs[i][j], eff_diffs[i][j])
                ax_s[i][j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (-1, -0.7))

                ax_s[i][j].set_xlim(-10, 10)
                ax_s[i][j].set_ylim(-1.05, 1.05)


                ax_s[i][j].set_xticks(np.arange(-10, 10, step=1))
                ax_s[i][j].set_xticklabels(np.arange(-10, 10, step=1))

                ax_s[i][j].set_yticks(np.arange(-1, 1.1, step=0.1))
                ax_s[i][j].set_yticklabels(np.arange(-1, 1.1, step=0.1).round(decimals=1))

            
    fig_s.show()
    
    if is_counts:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_cnts_all_diff.pdf', bbox_inches='tight')
    else:
        fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_rates_all_diff.pdf', bbox_inches='tight')
    

### Structural features vs effeciencies (rep averages)

In [ ]:
expr_colnames = []
eff_colnames = []
str_colnames = ['dG','Number_of_A_around_5_prime','Number_of_U_around_5_prime','stem_length_left(nt)','Loop_length(nt)','stem_length_right(nt)','GC_content_of_stem_region(left)']


for phase in phases:
    expr_colnames.append(phase+'_avg_norm_expr')
    
    if eff_measure == 'efficiency':
        eff_colnames.append(phase+'_avg_eff')
    elif eff_measure == 'readthrough':
        eff_colnames.append(phase+'_rt_avg')
    elif eff_measure == 'reinitiation':
        eff_colnames.append(phase+'_ri_avg')
    elif eff_measure == 'termination':
        eff_colnames.append(phase+'_t_avg')
    else: 
        raise TypeError('Unknown eff measure: '+str(eff_measure))

                      
fig_s, ax_s = plt.subplots(len(phases),len(str_colnames))
fig_s.set_size_inches(7*len(str_colnames),7*len(phases))

fig_s.suptitle('Terminator '+eff_measure+' vs. structural features: '+ experiment_set+', replicate averages')

for i in range(len(phases)):
    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    for j in range(len(str_colnames)):
        str_colname = str_colnames[j]
    
        eff_colname = eff_colnames[i]
        # Drop NA values (and ones?)
        curr_final_terms_df = final_terms_df[[expr_colname, str_colname, eff_colname]].dropna(how='any')

        # Delelte eff = 1?
        if is_eff_1_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]

        # Delelte eff = 0?
        if is_eff_0_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]

        # Delete negative efficiencies?
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]

        # Delete efficiencises > max_eff_th
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]


        # Delete expressions < treshold
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]

        #expressions = curr_final_terms_df[expr_colname].tolist()


        str_features = curr_final_terms_df[str_colname].tolist()
        efficiencies = curr_final_terms_df[eff_colname].tolist()

        ax_s[i,j].set_title(eff_colname+' '+str_colname, fontsize=10)
        
        if j == 0 or j == len(str_colnames) - 1: # dG or GC content
            # Scatterplot
            ax_s[i,j].scatter(str_features, efficiencies, s=4)

            ax_s[i,j].set_xlabel(str_colname)
            ax_s[i,j].set_ylabel('Terminator '+eff_measure + ' rate')

            # Correlations

            r, p = scipy.stats.pearsonr(str_features, efficiencies)
            rho, prho = scipy.stats.spearmanr(str_features, efficiencies)
            tau, ptau = scipy.stats.kendalltau(str_features, efficiencies)
            #ax_s[i,j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (0.2, 280), xycoords='axes points')

            #ax_s[i,j].set_xlim(-0.5, max_expr_th+0.5)
            ax_s[i,j].set_ylim(-0.05, 1.05)


            #ax_s[i,j].set_xticks(np.arange(0, max_expr_th, step=0.5))
            #ax_s[i,j].set_xticklabels(np.arange(0, max_expr_th, step=0.5))

            ax_s[i,j].set_yticks(np.arange(0, 1.1, step=0.1))
            ax_s[i,j].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))
            
        else:
        
            # Box plots
            ## Prepare data for box plots
            boxplot_data = []
            #print(str_features)
            unique_str_ft_values = sorted(list(set(str_features)))
            #print(unique_str_ft_values)
            xlabels = []
            for ft_value in unique_str_ft_values:
                indeces = [idx for idx in range(len(str_features)) if str_features[idx] == ft_value]
                #print('indexes:')
                #print(indeces)
                effcs = [efficiencies[q] for q in indeces]
                xlabels.append(ft_value)
                boxplot_data.append(effcs)

            # Creating plot
            ax_s[i,j].set_xlabel(str_colname)
            ax_s[i,j].set_ylabel('Terminator '+eff_measure + ' rate')
            bp = ax_s[i,j].boxplot(boxplot_data,labels=xlabels,bootstrap=1000)
            
        # Correlations

        r, p = scipy.stats.pearsonr(str_features, efficiencies)
        rho, prho = scipy.stats.spearmanr(str_features, efficiencies)
        tau, ptau = scipy.stats.kendalltau(str_features, efficiencies)
        ax_s[i,j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (0.2, 280), xycoords='axes points')

            


fig_s.show()
fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_STR_phases.pdf', bbox_inches='tight')
       

### Structural features vs effeciencies (each rep and phase)

In [ ]:
rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

str_colnames = ['dG','Number_of_A_around_5_prime','Number_of_U_around_5_prime','stem_length_left(nt)','Loop_length(nt)','stem_length_right(nt)','GC_content_of_stem_region(left)']
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')        
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))

        
fig_s, ax_s = plt.subplots(rep_number,len(str_colnames))
fig_s.set_size_inches(7*len(str_colnames),7*rep_number)
fig_s.suptitle('Terminator '+eff_measure+' vs. structural features: '+ experiment_set+', each phase & replicate seperately')

for i in range(rep_number):
    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    for j in range(len(str_colnames)):
        str_colname = str_colnames[j]
    
        eff_colname = eff_colnames[i]
        # Drop NA values (and ones?)
        curr_final_terms_df = final_terms_df[[expr_colname, str_colname, eff_colname]].dropna(how='any')

        # Delelte eff = 1?
        if is_eff_1_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]

        # Delelte eff = 0?
        if is_eff_0_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]

        # Delete negative efficiencies?
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]

        # Delete efficiencises > max_eff_th
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]


        # Delete expressions < treshold
        if is_neg_eff_deleted:
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
            curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]

        #expressions = curr_final_terms_df[expr_colname].tolist()


        str_features = curr_final_terms_df[str_colname].tolist()
        efficiencies = curr_final_terms_df[eff_colname].tolist()

        ax_s[i,j].set_title(eff_colname+' '+str_colname, fontsize=10)
        
        if j == 0 or j == len(str_colnames) - 1: # dG or GC content
            # Scatterplot
            ax_s[i,j].scatter(str_features, efficiencies, s=4)

            ax_s[i,j].set_xlabel(str_colname)
            ax_s[i,j].set_ylabel('Terminator '+eff_measure + ' rate')

            # Correlations

            r, p = scipy.stats.pearsonr(str_features, efficiencies)
            rho, prho = scipy.stats.spearmanr(str_features, efficiencies)
            tau, ptau = scipy.stats.kendalltau(str_features, efficiencies)
            #ax_s[i,j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (0.2, 250), xycoords='axes points')

            #ax_s[i,j].set_xlim(-0.5, max_expr_th+0.5)
            ax_s[i,j].set_ylim(-0.05, 1.05)


            #ax_s[i,j].set_xticks(np.arange(0, max_expr_th, step=0.5))
            #ax_s[i,j].set_xticklabels(np.arange(0, max_expr_th, step=0.5))

            ax_s[i,j].set_yticks(np.arange(0, 1.1, step=0.1))
            ax_s[i,j].set_yticklabels(np.arange(0, 1.1, step=0.1).round(decimals=1))
            
        else:
        
            # Box plots
            ## Prepare data for box plots
            boxplot_data = []
            unique_str_ft_values = sorted(list(set(str_features)))
            xlabels = []
            for ft_value in unique_str_ft_values:
                indeces = [idx for idx in range(len(str_features)) if str_features[idx] == ft_value]
                effcs = [efficiencies[q] for q in indeces]
                xlabels.append(ft_value)
                boxplot_data.append(effcs)

            # Creating plot
            ax_s[i,j].set_xlabel(str_colname)
            ax_s[i,j].set_ylabel('Terminator '+eff_measure + ' rate')
            bp = ax_s[i,j].boxplot(boxplot_data,labels=xlabels,bootstrap=1000)
            
        # Correlations

        r, p = scipy.stats.pearsonr(str_features, efficiencies)
        rho, prho = scipy.stats.spearmanr(str_features, efficiencies)
        tau, ptau = scipy.stats.kendalltau(str_features, efficiencies)
        ax_s[i,j].annotate('    Person R = '+ str(round(r,1)) + ', p = '+ str(round(p,2))+ '\n    Spearman rho =  '+ str(round(rho,1))+ ', p = '+ str(round(prho,4))+ '\n    Kendall tau =  '+ str(round(tau,1))+ ', p = '+ str(round(ptau,4)), (0.2, 280), xycoords='axes points')

            


fig_s.show()
fig_s.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_STR_reps_phases.pdf', bbox_inches='tight')
       

### Export fasta file with all terminators for blast-based overlap with experimental ones

In [ ]:
# All terminators before filtration (from terms_df)
fasta_writer = open(all_Ju_terms_fasta_file_path, 'w')
for index, row in terms_df.iterrows():
    fasta_writer.write('> ' + str(row['TTS_position'])+'_'+row['TTS_strand'] + '\n')
    fasta_writer.write(row["TTS_sequence(include_8nt_of_each_flank_sequence)"].replace('U','T') + '\n')
fasta_writer.close()


### Termination vs redthrough vs reinitiation for high and low expressed genes

In [ ]:
low_high_expr_th = 1

rep_cnt = 0

fig = plt.figure(figsize=(20, 10))

#fig, ax = plt.subplots()
fig.suptitle("Average proportion of each termination mode depends on gene expression level", fontsize=16)
for phase in phases:
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
        
    for rep in reps:
        rep_cnt += 1

        # Import data
        term_mode_df = final_terms_df[['TTS_id', phase+'_'+rep+'_pc_t', phase+'_'+rep+'_pc_rt', phase+'_'+rep+'_pc_ri', phase+'_'+rep+'_norm_expr']]

        # Drop terminators with zero trancripts around
        term_mode_df = term_mode_df.dropna(how='any')
        
        term_mode_h_expr = term_mode_df[term_mode_df[phase+'_'+rep+'_norm_expr'] >= low_high_expr_th]
        term_mode_l_expr = term_mode_df[term_mode_df[phase+'_'+rep+'_norm_expr'] < low_high_expr_th]
        
        t_h = term_mode_h_expr[phase+'_'+rep+'_pc_t'].tolist()
        rt_h = term_mode_h_expr[phase+'_'+rep+'_pc_rt'].tolist()
        ri_h = term_mode_h_expr[phase+'_'+rep+'_pc_ri'].tolist()
        
        t_l = term_mode_l_expr[phase+'_'+rep+'_pc_t'].tolist()
        rt_l = term_mode_l_expr[phase+'_'+rep+'_pc_rt'].tolist()
        ri_l = term_mode_l_expr[phase+'_'+rep+'_pc_ri'].tolist()
        
        #print(t_l)
        
        t_h_avg =  sum(t_h) / len(t_h)
        rt_h_avg =  sum(rt_h) / len(rt_h)
        ri_h_avg =  sum(ri_h) / len(ri_h)
        
        t_l_avg =  sum(t_l) / len(t_l)
        rt_l_avg =  sum(rt_l) / len(rt_l)
        ri_l_avg =  sum(ri_l) / len(ri_l)

        species = (
            "Low expression",
            "High expression"
        )
        weight_counts = {
            "Terminated": np.array([t_l_avg, t_h_avg]),
            "Read-through": np.array([rt_l_avg, rt_h_avg]),
            "Reinitiated": np.array([ri_l_avg, ri_h_avg])
        }
        width = 0.5

        ax = fig.add_subplot(2, 5, rep_cnt)
        
        bottom = np.zeros(2)
        for boolean, weight_count in weight_counts.items():
            p = ax.bar(species, weight_count, width, label=boolean, bottom=bottom)
            bottom += weight_count

        ax.set_title(phase+'_'+rep)
        ax.legend(loc="lower center")

plt.show()
fig.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_mode_proprtion.pdf', bbox_inches='tight')

### Readthrough efficiency and reinitiation eff for high and low expressed genes

In [ ]:
rep_cnt = 0

fig = plt.figure(figsize=(20, 10))

#fig, ax = plt.subplots()
fig.suptitle('Readthrough rate distribution depends on expression level', fontsize=16)
for phase in phases:
    
    if phase == 'log':
        reps = log_reps
    elif phase == 'stat':
        reps = stat_reps
        
    for rep in reps:
        rep_cnt += 1

        # Import data
        term_mode_df = final_terms_df[['TTS_id', phase+'_'+rep+'_pc_t', phase+'_'+rep+'_pc_rt', phase+'_'+rep+'_pc_ri', phase+'_'+rep+'_norm_expr']]

        # Drop terminators with zero trancripts around
        term_mode_df = term_mode_df.dropna(how='any')
        
        term_mode_h_expr = term_mode_df[term_mode_df[phase+'_'+rep+'_norm_expr'] >= low_high_expr_th]
        term_mode_l_expr = term_mode_df[term_mode_df[phase+'_'+rep+'_norm_expr'] < low_high_expr_th]
        
        t_h = term_mode_h_expr[phase+'_'+rep+'_pc_t'].tolist()
        rt_h = term_mode_h_expr[phase+'_'+rep+'_pc_rt'].tolist()
        ri_h = term_mode_h_expr[phase+'_'+rep+'_pc_ri'].tolist()
        
        before_tr_h_with0 = [m+n for m, n in zip(t_h, rt_h)] # all trancripts in 'before' window
        
        rt_eff_h = []
        ri_eff_h = []
        for i in range(len(before_tr_h_with0)):
            before = before_tr_h_with0[i]
            if before == 0:
                continue
            else:
                rt_eff_h.append(rt_h[i]/before)
                if ri_h[i]/before <= 1:
                    ri_eff_h.append(ri_h[i]/before)
                
        t_l = term_mode_l_expr[phase+'_'+rep+'_pc_t'].tolist()
        rt_l = term_mode_l_expr[phase+'_'+rep+'_pc_rt'].tolist()
        ri_l = term_mode_l_expr[phase+'_'+rep+'_pc_ri'].tolist()
        
        before_tr_l_with0 = [m+n for m, n in zip(t_l, rt_l)] # all trancripts in 'before' window
            
        rt_eff_l = []
        ri_eff_l = []       
        for i in range(len(before_tr_l_with0)):
            before = before_tr_l_with0[i]
            if before == 0:
                continue
            else:
                rt_eff_l.append(rt_l[i]/before)
                if ri_l[i]/before <= 1:
                    ri_eff_l.append(ri_l[i]/before)        
        

        ax = fig.add_subplot(2, 5, rep_cnt)
        
        ax.set_title('Default violin plot')
        ax.set_ylabel('Readthrough rate')
        ax.set_xlabel('Expression level')
        ax.violinplot([rt_eff_l, rt_eff_h], showmeans=True)
        ax.set_xticklabels(['','low', '', 'high'])
        #ax.set_xticklabels(['low', 'high'])
        ax.set_title(phase+'_'+rep)
        ax.legend(loc="lower center")

plt.show()
fig.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_mode_violin.pdf', bbox_inches='tight')

### PCA

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.preprocessing import StandardScaler


rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

expr_colnames = []
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')       
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
   
            
           
#fig_s, ax_s = plt.subplots(rep_number,1)
#fig_s.set_size_inches(7*1,7*rep_number)
#fig_s.suptitle('Terminator '+eff_measure+' vs. expression: '+ experiment_set+', each phase & replicate seperately')

for i in range(rep_number):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        
    #print(curr_final_terms_df.columns[:30])

        
    #ax_s[i].set_title(eff_colname, fontsize=10)
    
    #++++++++++++
    


    df = curr_final_terms_df
    #str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    X = df[features]

    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    pca = PCA(n_components=2)
    components = pca.fit_transform(X_std)

    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

    fig = px.scatter(components, x=0, y=1, color=np.log10(df[eff_colname]))

    for i, feature in enumerate(features):
        fig.add_annotation(
            ax=0, ay=0,
            axref="x", ayref="y",
            x=loadings[i, 0],
            y=loadings[i, 1],
            showarrow=True,
            arrowsize=2,
            arrowhead=2,
            xanchor="right",
            yanchor="top"
        )
        fig.add_annotation(
            x=loadings[i, 0],
            y=loadings[i, 1],
            ax=0, ay=0,
            xanchor="center",
            yanchor="bottom",
            text=feature,
            yshift=5,
        )
    fig.show()

#fig.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_PCA_reps_phases.pdf', bbox_inches='tight')
       

### Linear regression

In [ ]:
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression

rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

expr_colnames = []
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')       
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
   
            
           
#fig_s, ax_s = plt.subplots(rep_number,1)
#fig_s.set_size_inches(7*1,7*rep_number)
#fig_s.suptitle('Terminator '+eff_measure+' vs. expression: '+ experiment_set+', each phase & replicate seperately')

for i in range(rep_number):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        

    
    #++++++++++++
    

    str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    #str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    
    # Load your dataset (replace 'data.csv' with your actual dataset)
    data = curr_final_terms_df

    # Split the dataset into features (X) and target variable (y)
    X = data[features]
    y = data[eff_colname]

    # Split the data into training and testing sets
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train = X
    y_train = y
    
    # Train the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the test set
    #y_pred = model.predict(X_test)

    # Bootstrap to estimate confidence intervals for feature coefficients
    n_bootstraps = 1000
    bootstrap_coefs = []

    for _ in range(n_bootstraps):
        #X_boot, X_test_boot, y_boot, y_test_boot = train_test_split(X, y, test_size=0.2)#, random_state=42)
        X_boot, y_boot = resample(X_train, y_train)
        model_b = LinearRegression()
        model_b.fit(X_boot, y_boot)
        bootstrap_coefs.append(model_b.coef_)

    # Calculate confidence intervals
    confidence_intervals = np.percentile(bootstrap_coefs, [25, 75], axis=0)
    #print(confidence_intervals)
    
    # Calculate medians as central value estimators
    #print(bootstrap_importances)
    median_importances = np.median(bootstrap_coefs, axis=0)
    #print(median_importances)
    #print(model.feature_importances_)

    # Get feature importance
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': median_importances, 'ci_l': confidence_intervals[0], 'ci_h': confidence_intervals[1]})

    # Sort features by importance
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Visualize feature importance with confidence intervals
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='skyblue')
    plt.errorbar(x=feature_importance['Importance'], y=feature_importance['Feature'],
                 xerr=[feature_importance['Importance'] - feature_importance['ci_l'], feature_importance['ci_h'] - feature_importance['Importance']],
                 fmt='o', color='black', capsize=5, capthick=2)
    plt.xlabel('Coefficient')
    plt.ylabel('Feature')
    plt.title('Feature Coefficients with IQRs')
    plt.show()

    # Display feature importance table with confidence intervals
    print('\nFeature Coefficients with IQRs:')
    print(feature_importance)

### Random forest

#### RF: reps & phases

In [ ]:
rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

expr_colnames = []
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')        
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
   
            
           
fig_rf, ax_rf = plt.subplots(rep_number,1)
fig_rf.set_size_inches(10,7*rep_number)
fig_rf.suptitle('Feature Importance (Random Forest) with IQR: each phase & replicate seperately')

for i in range(rep_number):
    

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        
    
    #++++++++++++
    


    str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    #str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    
    # Load your dataset (replace 'data.csv' with your actual dataset)
    data = curr_final_terms_df
    
    X = data[features]
    y = data[eff_colname]

    # Split the data into training and testing sets
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X
    y_train = y
    
    # Train the Random Forest regression model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    #y_pred = model.predict(X_test)

    # Bootstrap to estimate confidence intervals for feature importances
    n_bootstraps = 1000
    bootstrap_importances = []

    for _ in range(n_bootstraps):
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        X_boot, y_boot = resample(X_train, y_train)
        model.fit(X_boot, y_boot)
        bootstrap_importances.append(model.feature_importances_)

    # Calculate confidence intervals
    confidence_intervals = np.percentile(bootstrap_importances, [25, 75], axis=0)
    #print(confidence_intervals)
    
    # Calculate medians as central value estimators
    #print(bootstrap_importances)
    median_importances = np.median(bootstrap_importances, axis=0)
    #print(median_importances)
    #print(model.feature_importances_)

    # Get feature importance
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': median_importances, 'ci_l': confidence_intervals[0], 'ci_h': confidence_intervals[1]})

    # Sort features by importance
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Visualize feature importance with confidence intervals
    ax_rf[i].set_title(all_reps_phases[i], fontsize=10)
    ax_rf[i].barh(feature_importance['Feature'], feature_importance['Importance'], color='skyblue')
    
    ax_rf[i].errorbar(x=feature_importance['Importance'], y=feature_importance['Feature'],
                 xerr=[feature_importance['Importance'] - feature_importance['ci_l'], feature_importance['ci_h'] - feature_importance['Importance']],
                 fmt='o', color='black', capsize=5, capthick=2)
    ax_rf[i].set_xlabel('Importance')
    ax_rf[i].set_ylabel('Feature')

fig_rf.show()
fig_rf.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_RF_reps_phases.pdf', bbox_inches='tight')

#### RF: phases (rep avereges)

In [ ]:
eff_colnames = []
expr_colnames = []
for phase in phases:
    expr_colnames.append(phase+'_avg_norm_expr')

    if eff_measure == 'efficiency':
        eff_colnames.append(phase+'_avg_eff')
    elif eff_measure == 'readthrough':
        eff_colnames.append(phase+'_rt_avg')
    elif eff_measure == 'reinitiation':
        eff_colnames.append(phase+'_ri_avg')
    elif eff_measure == 'termination':
        eff_colnames.append(phase+'_t_avg')
    else: 
        raise TypeError('Unknown eff measure: '+str(eff_measure))
            

fig_rf, ax_rf = plt.subplots(len(phases),1)
fig_rf.set_size_inches(10,7*len(phases))
fig_rf.suptitle('Feature Importance (Random Forest) with IQR: phase averages')
            
for i in range(len(phases)):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        

        
    #ax_s[i].set_title(eff_colname, fontsize=10)
    
    #++++++++++++
    


    str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    #str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    #print(expr_colname)
    # Load your dataset (replace 'data.csv' with your actual dataset)
    data = curr_final_terms_df[features + [eff_colname]]
    
    X = data[features]
    y = data[eff_colname]
    #display(data.head())

    # Split the data into training and testing sets
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X
    y_train = y
    
    # Train the Random Forest regression model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    #y_pred = model.predict(X_test)

    # Bootstrap to estimate confidence intervals for feature importances
    n_bootstraps = 1000
    bootstrap_importances = []

    for _ in range(n_bootstraps):
        X_boot, y_boot = resample(X_train, y_train)
        model.fit(X_boot, y_boot)
        bootstrap_importances.append(model.feature_importances_)

    # Calculate confidence intervals
    confidence_intervals = np.percentile(bootstrap_importances, [25, 75], axis=0)
    
    # Calculate medians as central value estimators
    median_importances = np.median(bootstrap_importances, axis=0)

    # Get feature importance
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': median_importances, 'ci_l': confidence_intervals[0], 'ci_h': confidence_intervals[1]})

    # Sort features by importance
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Visualize feature importance with confidence intervals
    ax_rf[i].set_title(phases[i], fontsize=10)
    ax_rf[i].barh(feature_importance['Feature'], feature_importance['Importance'], color='skyblue')
    
    ax_rf[i].errorbar(x=feature_importance['Importance'], y=feature_importance['Feature'],
                 xerr=[feature_importance['Importance'] - feature_importance['ci_l'], feature_importance['ci_h'] - feature_importance['Importance']],
                 fmt='o', color='black', capsize=5, capthick=2)
    ax_rf[i].set_xlabel('Importance')
    ax_rf[i].set_ylabel('Feature')

fig_rf.show()
fig_rf.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_RF_phases.pdf', bbox_inches='tight')

### Partial correlation

#### PC: reps & phases

In [ ]:
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
import pingouin as pg

rep_number = len(log_reps) + len(stat_reps)
all_reps_phases = log_reps + stat_reps

expr_colnames = []
eff_colnames = []

for phase in phases:
    if phase == 'log':
        for rep in log_reps:
            if is_expr_norm:
                expr_colnames.append('log_'+rep+'_norm_expr')       
                            
            if eff_measure == 'efficiency':
                eff_colnames.append('log_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('log_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('log_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('log_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
                
    elif phase == 'stat':
        for rep in stat_reps:
            if is_expr_norm:
                expr_colnames.append('stat_'+rep+'_norm_expr')
                
            if eff_measure == 'efficiency':
                eff_colnames.append('stat_'+rep+'_eff')
            elif eff_measure == 'readthrough':
                eff_colnames.append('stat_'+rep+'_pc_rt')
            elif eff_measure == 'reinitiation':
                eff_colnames.append('stat_'+rep+'_pc_ri')
            elif eff_measure == 'termination':
                eff_colnames.append('stat_'+rep+'_pc_t')
            else: 
                raise TypeError('Unknown eff measure: '+str(eff_measure))
   

fig_pc, ax_pc = plt.subplots(rep_number,1)
fig_pc.set_size_inches(10,6*rep_number)
fig_pc.suptitle('Partial correlation of each terminator feature and readthrough rate (with IQRs): each phase & replicate seperately')

for i in range(rep_number):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        
    #print(curr_final_terms_df.columns[:30])

        
    #ax_s[i].set_title(eff_colname, fontsize=10)
    
    #++++++++++++

    str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    #str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    
    # Load your dataset (replace 'data.csv' with your actual dataset)
    data = curr_final_terms_df[features + [eff_colname]]

    # Compute partial correlation coefficients between each predictor variable and the target variable
    partial_correlations = {}
    for column in data.columns:
        if column in features:
            # Bootstrap to estimate confidence intervals for partial correlation coefficients
            partial_corrs_bootstrap = []
            for _ in range(1000):
                boot_sample = resample(data, replace=True)
                partial_corr = pg.partial_corr(data=boot_sample, x=column, y=eff_colname, covar=data.drop(columns=[column, eff_colname]).columns.tolist())
                partial_corrs_bootstrap.append(partial_corr['r'].values[0])
                #print(partial_corrs_bootstrap)
            # Calculate confidence intervals
            ci = np.percentile(partial_corrs_bootstrap, [25, 75])
            partial_correlations[column] = {'Partial Correlation': pg.partial_corr(data=data, x=column, y=eff_colname, covar=data.drop(columns=[column, eff_colname]).columns.tolist())['r'].values[0], 'CI Lower': ci[0], 'CI Upper': ci[1]}

    # Convert dictionary to DataFrame
    partial_correlations_df = pd.DataFrame(partial_correlations).T.reset_index().rename(columns={'index': 'Feature'})

    # Sort features by absolute partial correlation coefficient
    partial_correlations_df['Absolute Partial Correlation'] = np.abs(partial_correlations_df['Partial Correlation'])
    partial_correlations_df = partial_correlations_df.sort_values(by='Absolute Partial Correlation', ascending=False).drop(columns='Absolute Partial Correlation')

    # Visualize partial correlation coefficients with confidence intervals
    ax_pc[i].barh(partial_correlations_df['Feature'], partial_correlations_df['Partial Correlation'], color='skyblue')
    ax_pc[i].errorbar(x=partial_correlations_df['Partial Correlation'], y=partial_correlations_df['Feature'],
                 xerr=[partial_correlations_df['Partial Correlation'] - partial_correlations_df['CI Lower'], partial_correlations_df['CI Upper'] - partial_correlations_df['Partial Correlation']],
                 fmt='o', color='black', capsize=5, capthick=2)
    ax_pc[i].set_xlabel('Partial Correlation')
    ax_pc[i].set_ylabel('Feature')

fig_pc.show()
fig_pc.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_PC_reps_phases.pdf', bbox_inches='tight')

#### PC: phases (rep avereges)

In [ ]:
eff_colnames = []
expr_colnames = []
for phase in phases:
    expr_colnames.append(phase+'_avg_norm_expr')

    if eff_measure == 'efficiency':
        eff_colnames.append(phase+'_avg_eff')
    elif eff_measure == 'readthrough':
        eff_colnames.append(phase+'_rt_avg')
    elif eff_measure == 'reinitiation':
        eff_colnames.append(phase+'_ri_avg')
    elif eff_measure == 'termination':
        eff_colnames.append(phase+'_t_avg')
    else: 
        raise TypeError('Unknown eff measure: '+str(eff_measure))

fig_pc, ax_pc = plt.subplots(len(phases),1)
fig_pc.set_size_inches(10,6*len(phases))
fig_pc.suptitle('Partial correlation of each terminator feature and readthrough rate (with IQRs): phase averages')


for i in range(len(phases)):

    expr_colname = expr_colnames[i]
    eff_colname = eff_colnames[i]
    # Drop NA values (and ones?)
    curr_final_terms_df = final_terms_df.dropna(how='any', subset=[expr_colname, eff_colname])
    
    # Delelte eff = 1?
    if is_eff_1_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=1]
        
    # Delelte eff = 0?
    if is_eff_0_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname]!=0]
    
    # Delete negative efficiencies?
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] >= 0]
        
    # Delete efficiencises > max_eff_th
    curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[eff_colname] <= max_eff_th]
    
        
    # Delete expressions < treshold
    if is_neg_eff_deleted:
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] >= min_expr_th]
        curr_final_terms_df = curr_final_terms_df[curr_final_terms_df[expr_colname] <= max_expr_th]
        
    #print(curr_final_terms_df.columns[:30])

        
    #ax_s[i].set_title(eff_colname, fontsize=10)
    
    #++++++++++++

    str_features = ['dG', 'Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','stem_length_left(nt)', 'Loop_length(nt)', 'GC_content_of_stem_region(left)']
    #str_features = ['Number_of_A_around_5_prime', 'Number_of_U_around_5_prime','Loop_length(nt)']
    features =  [expr_colname] + str_features
    
    # Load your dataset (replace 'data.csv' with your actual dataset)
    data = curr_final_terms_df[features + [eff_colname]]
    #display(data.head())

    # Compute partial correlation coefficients between each predictor variable and the target variable
    partial_correlations = {}
    for column in data.columns:
        if column in features:
            # Bootstrap to estimate confidence intervals for partial correlation coefficients
            partial_corrs_bootstrap = []
            for _ in range(1000):
                boot_sample = resample(data, replace=True)
                partial_corr = pg.partial_corr(data=boot_sample, x=column, y=eff_colname, covar=data.drop(columns=[column, eff_colname]).columns.tolist())
                partial_corrs_bootstrap.append(partial_corr['r'].values[0])
                #print(partial_corrs_bootstrap)
            # Calculate confidence intervals
            ci = np.percentile(partial_corrs_bootstrap, [25, 75])
            partial_correlations[column] = {'Partial Correlation': pg.partial_corr(data=data, x=column, y=eff_colname, covar=data.drop(columns=[column, eff_colname]).columns.tolist())['r'].values[0], 'CI Lower': ci[0], 'CI Upper': ci[1]}

    # Convert dictionary to DataFrame
    partial_correlations_df = pd.DataFrame(partial_correlations).T.reset_index().rename(columns={'index': 'Feature'})

    # Sort features by absolute partial correlation coefficient
    partial_correlations_df['Absolute Partial Correlation'] = np.abs(partial_correlations_df['Partial Correlation'])
    partial_correlations_df = partial_correlations_df.sort_values(by='Absolute Partial Correlation', ascending=False).drop(columns='Absolute Partial Correlation')

    # Visualize partial correlation coefficients with confidence intervals
    ax_pc[i].barh(partial_correlations_df['Feature'], partial_correlations_df['Partial Correlation'], color='skyblue')
    ax_pc[i].errorbar(x=partial_correlations_df['Partial Correlation'], y=partial_correlations_df['Feature'],
                 xerr=[partial_correlations_df['Partial Correlation'] - partial_correlations_df['CI Lower'], partial_correlations_df['CI Upper'] - partial_correlations_df['Partial Correlation']],
                 fmt='o', color='black', capsize=5, capthick=2)
    ax_pc[i].set_xlabel('Partial Correlation')
    ax_pc[i].set_ylabel('Feature')

fig_pc.show()
fig_pc.savefig(str(plots_out_folder)+'/'+term_mech_prefix+'_'+bidir_prefix+'_'+eff_measure+'_PC_phases.pdf', bbox_inches='tight')